# 概念分群[二]_使用範例
LastModified : 20160117_2135

＠選定要使用的W2V Model (只能擇一，頗占記憶體)

In [1]:
import gensim
#1__WIKI
model = gensim.models.Word2Vec.load("W2V_Wiki_Trastd/wiki.zh.sim.seg.trastd.model")

#2__搜狐新闻数据
#model = gensim.models.Word2Vec.load('W2V_SogouCS/SogouCS_Combine.model')

#3__全网新闻数据
#model = gensim.models.Word2Vec.load('W2V_SogouCA/SogouCA_Combine.model')

In [2]:
def W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent):
    #SimLevel = 0.25#　　相似程度
    #SimPercent = 0.2#　　相似比率

    import operator
    cnt = []#　　計算餵進來的總詞量
    Expt = []#　　存W2V無法辨識的詞語
    Remain = []#　　存沒有通過門檻的詞語
    ClassingResult = {}

    for z in Concept:#　　把概念標頭寫進字典
        ClassingResult[z] = {}#　　每個字典的內容都是一個字典

    for q in open(Corpus,'r'):#　　把萃取關鍵字的結果讀出來        
        q = q.strip().decode('utf-8')#　　去空白,解碼
        cnt.append(q)#　　把讀取內容寫進一陣列(計算樣本的資料量用)
        try:
            TmpDic = {}#　　暫時性的字典(比對詞語與各概念的相似度)
            SameConcept = ''#　　用來存放概念,當q直接是某概念時使用
            TmpWord = 'None'#　　用來比對詞語是不是與概念完全吻合,若是,None會被修改掉;反之則維持None(初值)

            print '@ ->「',q,'」的概念分類開始: '

            for a in Concept:#　　讀取各概念標頭
                TmpList = []#　　暫時性的陣列(存放'詞語'vs'概念內各詞語'的相似度)     
                for b in Concept[a]:#　　讀取個概念內的詞語

                    #####完全ㄧ樣#####
                    if q.encode('utf-8') == b:#　　如果等於
                        print '---目標詞「',q,'」就在【',a,'】的概念裡'
                        TmpWord = b#　　把詞語指定給TmpWord
                        break#　　跳出迴圈(已經知道跟某詞語完全一樣,就不用再比其他的詞語了)
                    #####

                    if model.similarity(q,b.decode('utf-8')) > SimLevel:#　　如果相似度達門檻
                        TmpList.append(model.similarity(q,b.decode('utf-8')))#　　就把相似度寫進TmpList裡面

                #####完全一樣#####        
                if TmpWord != 'None':#　　不是None(初值)->被修改過->要完全相等才會出現此情況
                    SameConcept = a#　　把現在比對到的概念( EX:輕鬆 )記起來
                    break#　　跳出迴圈(已經知道是屬於哪個概念,就不用再比對其他概念了)
                #####

                #####印出q與一概念比較的詳細數據
                print '  -目標詞「',q,'」與【',a,'】的相似陣列( 顯示內容為已高於相似度門檻 :',SimLevel,')'
                print TmpList
                print '  -其占【',a,'】概念內容的\n   比重為 :',len(TmpList),'/',len(Dic[a])
                print '   比率為 :',
                print float(len(TmpList))/float(len(Concept[a]))
                print '*'
                #####

                if float(len(TmpList))/float(len(Concept[a])) > SimPercent:#　　如果達到相似成數的門檻
                    print '  -「',q,'」已過相似_百分比_門檻 :',SimPercent
                    TmpAvg = 0.0#　　存放平均相似度
                    for c in TmpList:#　　把每個相似度讀出來
                        TmpAvg += c#　　加起來
                    TmpDic[a] = float(TmpAvg)/float(len(TmpList))#　　把'總數'除以'陣列長度',得到平均相似度,
                                                                    #再寫進字典裡,當q與多個概念都有相似度食
                                                                    #可以比較與哪個概念的相似度比較高
                    print '   與【',a,'】的相似度為: ',TmpDic[a],' -> 進入候選名單'
                    print '# # # # # # #\n'

            #####完全一樣##### 
            if TmpWord != 'None': 
                ClassingResult[SameConcept][(q.encode('utf-8'))] = 1#　　把q寫進總字典的概念裡(Key:q Value:1)
                continue
            #####

            #####檢視暫存字典#####
            print '\n___檢視',q,'的暫存字典(TmpDic)___'
            for k in TmpDic:#　　遍歷暫存字典,看q與個概念的相似度
                print k,TmpDic[k]
            print '== == == == == == == == == == =='
            #####

            if len(TmpDic) > 0:#　　如果暫存字典有任何一概念(表示q'至少'與一個概念相似)
                print '> >小結 :',q,'成功歸類！',
                for d in TmpDic:#　　遍歷TmpDic
                    if d == sorted(TmpDic.iteritems(),key=operator.itemgetter(1),reverse=True)[0][0]:
                        #如果d = 排序後的第一名 (表示d就是與q最像的概念)
                        if TmpDic[d] > SimLevel:#　　如果"q與概念的相似度"大於相似度門檻
                            print '類別為 :',d,'\n'
                            ClassingResult[d][q.encode('utf-8')] = TmpDic[d]#　　把q寫進總字典的概念裡(Key:q Value:相似度)
                        else:#　　不太可能發生的情況
                            Remain.append('WHAT!?'+q.encode('utf-8'))                    
            else:#　　q與各概念相似度都沒有達標,無法歸類
                Remain.append(q.encode('utf-8'))

        except KeyError:#　　抓到KeyError( W2V不認識q )的例外處理
            print '「',q,'」EXCEPT! (KeyError)\n'
            Expt.append(q.encode('utf-8'))#　　把q加進例外的陣列理

    ###########檢視總字典#####
    print '\n+ + + 印出字典看看 + + +'
    for ee in ClassingResult:
        print ee,str(ClassingResult[ee]).decode('string_escape')
    print '+ + + + + + + + + + + +'
    ###########

    print '原總量',len(cnt),'\t|','去重複後',len(set(cnt))
    print '--------------------------------------------------'        

    Scnt = 0#　　計算成功被歸類的數量
    for e in ClassingResult:#　　遍歷各概念
        print '【',e,'】','\t數量:',len(ClassingResult[e])
        for f in ClassingResult[e]:#　　遍歷被歸類到此概念中的詞語　
            print f,#　　若加ClassingResult[e][f]則會印出詞語與概念的相似度
            Scnt += 1
        print '\n'

    print '--------------------------------------------------------------------'
    print '相似度門檻',SimLevel,'\t|','相似成數門檻',SimPercent,'　|',\
          '成功分類',Scnt,'　|','成功分類比率',int(float(Scnt)/float(len(set(cnt)))*100),'%'
    print '--------------------------------------------------------------------\n'

    #在此之前Expt 和 Remain都是List , 都有照順序排好 , 印出來可檢查這些詞的原順序
    #此時把List轉成Set , 去除重複值 , 增加可讀性
    Expt = set(Expt)
    Remain = set(Remain)

    #把W2V不認識的印出來
    print '【W2V無法辨識】\t數量',len(Expt)
    for g in Expt:
        print g,
    print '\n'

    #把沒過門檻值的印出來
    print '【低於門檻值】\t數量',len(Remain)
    for h in Remain:
        print h,
    print '\n'

# 將詞語分類

使用方法：

    W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent)
    
    Concept   :[字典]。概念字典。
               格式為：Key  : 概念的名稱(僅命名,無實質意義)
               Value: [陣列],內容是可以代表該概念的詞語
               
    Corpus    :[字串]。[檔名]。存取關鍵字的TXT檔。
                注意！檔案內第一行要空著！
    
    SimLevel  :[浮點數]。[0~1]。相似度門檻。
    
    SimPercent:[浮點數]。[0~1]。相似成數(百分比)門檻。
                必須與概念內詞語相似達到一定程度才能算與此概念相似。

# 實際測試
(PS 操作時都會多一個空白, 所以會多一個詞, 是正常現象)

VVV測試用概念字典VVV

In [3]:
Dic = {'脫俗':['脫俗','幽雅','清幽','優雅','氣質','心曠神怡'],\
       '輕鬆':['輕鬆','休閒','遊憩','悠閒','休憩','消遣','愜意','放鬆']}

# 測試[A] - 詞語跟概念中詞語完全相似
詞　　語: [ 輕鬆 ]

相似門檻: 0.25　　|　相似成數: 0.2

In [4]:
W2V_ClassificationMethod2(Dic,'test0114.txt',0.25,0.2)

@ ->「 ﻿ 」的概念分類開始: 
「 ﻿ 」EXCEPT! (KeyError)

@ ->「 輕鬆 」的概念分類開始: 
  -目標詞「 輕鬆 」與【 脫俗 】的相似陣列( 顯示內容為已高於相似度門檻 : 0.25 )
[0.30271165470057576]
  -其占【 脫俗 】概念內容的
   比重為 : 1 / 6
   比率為 : 0.166666666667
*
---目標詞「 輕鬆 」就在【 輕鬆 】的概念裡

+ + + 印出字典看看 + + +
脫俗 {}
輕鬆 {'輕鬆': 1}
+ + + + + + + + + + + +
原總量 2 	| 去重複後 2
--------------------------------------------------
【 脫俗 】 	數量: 0


【 輕鬆 】 	數量: 1
輕鬆 

--------------------------------------------------------------------
相似度門檻 0.25 	| 相似成數門檻 0.2 　| 成功分類 1 　| 成功分類比率 50 %
--------------------------------------------------------------------

【W2V無法辨識】	數量 1
﻿ 

【低於門檻值】	數量 0




# 測試[B] - 詞語與多個概念都有相似時
詞　　語: [ 悠哉 ]

相似門檻: 0.25　　|　相似成數: 0.2

In [5]:
W2V_ClassificationMethod2(Dic,'test0114.txt',0.25,0.2)

@ ->「 ﻿ 」的概念分類開始: 
「 ﻿ 」EXCEPT! (KeyError)

@ ->「 悠哉 」的概念分類開始: 
  -目標詞「 悠哉 」與【 脫俗 】的相似陣列( 顯示內容為已高於相似度門檻 : 0.25 )
[0.40213583263428443, 0.37660737395282673, 0.40546016760082743, 0.2617181126425962]
  -其占【 脫俗 】概念內容的
   比重為 : 4 / 6
   比率為 : 0.666666666667
*
  -「 悠哉 」已過相似_百分比_門檻 : 0.2
   與【 脫俗 】的相似度為:  0.361480371708  -> 進入候選名單
# # # # # # #

  -目標詞「 悠哉 」與【 輕鬆 】的相似陣列( 顯示內容為已高於相似度門檻 : 0.25 )
[0.31945289405416533, 0.42798378153279459, 0.33026052403947703]
  -其占【 輕鬆 】概念內容的
   比重為 : 3 / 8
   比率為 : 0.375
*
  -「 悠哉 」已過相似_百分比_門檻 : 0.2
   與【 輕鬆 】的相似度為:  0.359232399875  -> 進入候選名單
# # # # # # #


___檢視 悠哉 的暫存字典(TmpDic)___
脫俗 0.361480371708
輕鬆 0.359232399875
== == == == == == == == == == ==
> >小結 : 悠哉 成功歸類！ 類別為 : 脫俗 


+ + + 印出字典看看 + + +
脫俗 {'悠哉': 0.3614803717076337}
輕鬆 {}
+ + + + + + + + + + + +
原總量 2 	| 去重複後 2
--------------------------------------------------
【 脫俗 】 	數量: 1
悠哉 

【 輕鬆 】 	數量: 0


--------------------------------------------------------------------
相似度門檻 0.25 	| 相似成數門檻 0.2 　| 成功分類 1 　

# 測試[C] - 詞語是W2V無法辨識時
詞　　語: [ 肉形石 ]

相似門檻: 0.25　　|　相似成數: 0.2

In [6]:
W2V_ClassificationMethod2(Dic,'test0114.txt',0.25,0.2)

@ ->「 ﻿ 」的概念分類開始: 
「 ﻿ 」EXCEPT! (KeyError)

@ ->「 肉形石 」的概念分類開始: 
「 肉形石 」EXCEPT! (KeyError)


+ + + 印出字典看看 + + +
脫俗 {}
輕鬆 {}
+ + + + + + + + + + + +
原總量 2 	| 去重複後 2
--------------------------------------------------
【 脫俗 】 	數量: 0


【 輕鬆 】 	數量: 0


--------------------------------------------------------------------
相似度門檻 0.25 	| 相似成數門檻 0.2 　| 成功分類 0 　| 成功分類比率 0 %
--------------------------------------------------------------------

【W2V無法辨識】	數量 2
﻿ 肉形石 

【低於門檻值】	數量 0




# 測試[D1] - 詞語因相似度門檻而有所不同
詞　　語: [ 志趣 ]
相似門檻: 0.25　　|　相似成數: 0.2

In [7]:
W2V_ClassificationMethod2(Dic,'test0114.txt',0.25,0.2)

@ ->「 ﻿ 」的概念分類開始: 
「 ﻿ 」EXCEPT! (KeyError)

@ ->「 志趣 」的概念分類開始: 
  -目標詞「 志趣 」與【 脫俗 】的相似陣列( 顯示內容為已高於相似度門檻 : 0.25 )
[0.32413836781524286, 0.27796147861875337]
  -其占【 脫俗 】概念內容的
   比重為 : 2 / 6
   比率為 : 0.333333333333
*
  -「 志趣 」已過相似_百分比_門檻 : 0.2
   與【 脫俗 】的相似度為:  0.301049923217  -> 進入候選名單
# # # # # # #

  -目標詞「 志趣 」與【 輕鬆 】的相似陣列( 顯示內容為已高於相似度門檻 : 0.25 )
[]
  -其占【 輕鬆 】概念內容的
   比重為 : 0 / 8
   比率為 : 0.0
*

___檢視 志趣 的暫存字典(TmpDic)___
脫俗 0.301049923217
== == == == == == == == == == ==
> >小結 : 志趣 成功歸類！ 類別為 : 脫俗 


+ + + 印出字典看看 + + +
脫俗 {'志趣': 0.3010499232169981}
輕鬆 {}
+ + + + + + + + + + + +
原總量 2 	| 去重複後 2
--------------------------------------------------
【 脫俗 】 	數量: 1
志趣 

【 輕鬆 】 	數量: 0


--------------------------------------------------------------------
相似度門檻 0.25 	| 相似成數門檻 0.2 　| 成功分類 1 　| 成功分類比率 50 %
--------------------------------------------------------------------

【W2V無法辨識】	數量 1
﻿ 

【低於門檻值】	數量 0




# 測試[D2] - 詞語因相似度門檻而有所不同
詞　　語: [ 志趣 ]
相似門檻: 0.25　　|　相似成數: 0.6 (相似成數提高!!!)

In [8]:
W2V_ClassificationMethod2(Dic,'test0114.txt',0.25,0.6)

@ ->「 ﻿ 」的概念分類開始: 
「 ﻿ 」EXCEPT! (KeyError)

@ ->「 志趣 」的概念分類開始: 
  -目標詞「 志趣 」與【 脫俗 】的相似陣列( 顯示內容為已高於相似度門檻 : 0.25 )
[0.32413836781524286, 0.27796147861875337]
  -其占【 脫俗 】概念內容的
   比重為 : 2 / 6
   比率為 : 0.333333333333
*
  -目標詞「 志趣 」與【 輕鬆 】的相似陣列( 顯示內容為已高於相似度門檻 : 0.25 )
[]
  -其占【 輕鬆 】概念內容的
   比重為 : 0 / 8
   比率為 : 0.0
*

___檢視 志趣 的暫存字典(TmpDic)___
== == == == == == == == == == ==

+ + + 印出字典看看 + + +
脫俗 {}
輕鬆 {}
+ + + + + + + + + + + +
原總量 2 	| 去重複後 2
--------------------------------------------------
【 脫俗 】 	數量: 0


【 輕鬆 】 	數量: 0


--------------------------------------------------------------------
相似度門檻 0.25 	| 相似成數門檻 0.6 　| 成功分類 0 　| 成功分類比率 0 %
--------------------------------------------------------------------

【W2V無法辨識】	數量 1
﻿ 

【低於門檻值】	數量 1
志趣 

